# EP-14 Movielens 영화 추천 실습

## 진행 과정

1. 데이터 준비 및 전처리
2. 분석하기
    - ratings에 있는 유니크한 영화 개수
    - ratings에 있는 유니크한 사용자 수
    - 가장 인기 있는 영화 30개(인기순)
3. 내가 선호하는 영화 ratings에 추가하기
4. CSR matrix를 직접 만들어보기
5. 모델 구현 및 훈련
6. 선호도 파악 및 추천받기

### 데이터 준비 및 전처리

  - 이번에 활용하는 데이터셋은 Movielens 데이터입니다.
  - 이번 노드를 진행함에 있어서 몇 가지의 전제조건이 있습니다.
      1. 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
      2. 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
      3. 별점을 시청횟수로 해석해서 생각하겠습니다.
      4. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
  - Movielens 데이터는 인덱싱이 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# ratings 와 movies 데이터 프레임 합치기

data = ratings.join(movies.set_index('movie_id'), on='movie_id')
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하는 컬럼만 남겨줍니다.

using_cols = ['user_id', 'counts', 'title']
data = data[using_cols]
data.head(10)

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"
5,1,3,"Princess Bride, The (1987)"
6,1,5,Ben-Hur (1959)
7,1,5,"Christmas Story, A (1983)"
8,1,4,Snow White and the Seven Dwarfs (1937)
9,1,4,"Wizard of Oz, The (1939)"


### 분석하기

  - 유니크한 영화 개수 및 유니크한 사용자의 수, 가장 인기있는 영화 30개가 어떤 것이 있는지 분석해보겠습니다.

In [8]:
# 영화 개수

data['title'].nunique()

3628

In [9]:
# 사용자의 수

data['user_id'].nunique()

6039

In [10]:
# 가장 인기있는 영화 30개

movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 내가 선호하는 영화 ratings에 추가하기

In [11]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['My Fair Lady (1964)' , 'Terminator, The (1984)' ,'Men in Black (1997)' ,'Matrix, The (1999)' ,'Terminator 2: Judgment Day (1991)']

# 'kris'라는 user_id가 위 영화들을 10회씩 보았다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['kris']*5, 'title': my_favorite, 'counts':[10]*5})

if not data.isin({'user_id':['kris']})['user_id'].any():  # user_id에 'kris'이라는 데이터가 없다면
    data = data.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,counts,title
1000203,6040,3,Platoon (1986)
1000205,6040,5,"Crying Game, The (1992)"
1000206,6040,5,Welcome to the Dollhouse (1995)
1000207,6040,4,Sophie's Choice (1982)
1000208,6040,4,E.T. the Extra-Terrestrial (1982)
0,kris,10,My Fair Lady (1964)
1,kris,10,"Terminator, The (1984)"
2,kris,10,Men in Black (1997)
3,kris,10,"Matrix, The (1999)"
4,kris,10,Terminator 2: Judgment Day (1991)


In [12]:
# 모델에 활용하기 전 필요한 전처리 과정
#고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kris'])    
print(movie_to_idx['Matrix, The (1999)'])

6039
124


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,10,2
1,6039,10,200
2,6039,10,175
3,6039,10,124


### CSR matrix를 직접 만들어보기

  - 추천 시스템의 다양한 모델들 중에 Matrix Factorization(MF, 행렬 분해)모델을 사용해서 진행하겠습니다.

In [15]:
# CSR matrix

from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 모델 구현 및 학습 진행

  - Matrix Factorization 모델을 implicit 패키지를 사용하여 학습을 진행합니다
  - implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
  - 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. 

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

  - AlternatingLeastSquares 클래스의 __init__ 파라미터가 존재합니다.
      - factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
      - regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
      - use_gpu : GPU를 사용할 것인지
      - iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
  - factors와 iterations를 늘릴수록 과적합의 우려가 있기 때문에 조심해야 합니다.

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
# 훈련 확인 

kris, matrix = user_to_idx['kris'], movie_to_idx['Matrix, The (1999)']
kris_vector, matrix_vector = als_model.user_factors[kris], als_model.item_factors[matrix]

print('please')

please


In [21]:
kris_vector

array([-0.05479205,  0.23537889, -0.5483686 , -0.37734148,  0.9789008 ,
        1.0840018 ,  0.11328877,  0.28709066, -0.19557104,  0.7192768 ,
        0.07358733, -0.27922535, -1.0104487 ,  0.35855988, -0.25181827,
       -0.13321093, -0.67475605,  0.6936406 , -0.24680832, -0.30990538,
       -0.42603132, -0.32529593,  0.98779476,  0.32893714,  0.94586205,
       -0.3484653 , -0.15698677, -0.4007238 ,  1.3217214 ,  0.4185522 ,
       -0.4067778 ,  1.6034167 , -1.385972  , -0.06352438, -1.000149  ,
       -0.5253177 , -0.6675044 ,  0.6750362 ,  0.9621936 ,  0.21665949,
        0.63990045, -0.7639567 ,  0.65553236, -0.655749  ,  0.75473636,
        0.6315244 , -0.07952616, -0.21524827,  0.22085111, -0.8533679 ,
        0.4671943 , -0.71469074,  0.37815195, -0.52259207, -0.07329006,
        0.07054289,  0.27447683,  0.5069001 , -0.5330322 , -0.31244227,
       -0.41046607,  0.45104566, -1.5480548 , -1.023893  , -0.89772785,
       -0.7075105 ,  0.14002752,  1.0280579 , -0.40451416,  1.54

In [22]:
matrix_vector

array([ 4.19866480e-03,  1.61726251e-02,  4.20707371e-03, -1.18316999e-02,
        4.00861576e-02,  3.35878953e-02,  2.90474342e-03,  9.73203778e-03,
       -1.12765236e-02,  3.24625708e-02,  9.67152603e-03,  7.63198500e-03,
       -4.10779333e-03, -9.62561090e-03, -4.63568699e-03,  1.45566054e-02,
       -3.25974375e-02,  9.96205490e-03,  8.66447575e-03,  6.80869399e-03,
       -6.97143609e-04,  1.50760310e-02,  7.95367546e-03, -8.28326811e-05,
        2.20368654e-02, -9.74035531e-04, -1.04211196e-02, -2.14428455e-02,
        4.77577634e-02,  2.13046428e-02, -1.53210908e-02,  1.60628594e-02,
       -3.29885222e-02,  5.23861311e-03, -1.59650780e-02,  1.68105830e-02,
       -2.51029003e-02, -1.05067994e-03,  4.51535126e-03,  1.45685871e-03,
        3.59080620e-02,  5.92250936e-03,  1.90920681e-02, -3.39457707e-04,
        1.43945608e-02,  1.40454844e-02,  1.87175833e-02,  3.52452276e-03,
        1.97299141e-02, -1.43302074e-02,  1.55264763e-02, -9.87761002e-03,
        1.40497619e-02,  

In [23]:
# kris과 matrix를 내적하는 코드

np.dot(kris_vector, matrix_vector)

0.8198568

In [24]:
lady = movie_to_idx['My Fair Lady (1964)']
lady_vector = als_model.item_factors[lady]
np.dot(kris_vector, lady_vector)

0.40752292

### 선호도 파악 및 추천받기

In [28]:
# 비슷한 영화 찾아보기

favorite_movie = 'Matrix, The (1999)'
title = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(title, N=15)
similar_movie

[(124, 1.0000001),
 (92, 0.7745636),
 (62, 0.6678946),
 (141, 0.58436203),
 (200, 0.5777335),
 (145, 0.547041),
 (375, 0.53389776),
 (107, 0.522296),
 (317, 0.4971979),
 (175, 0.49576378),
 (44, 0.42375693),
 (75, 0.4159806),
 (236, 0.40229744),
 (117, 0.38552),
 (431, 0.38356978)]

In [29]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Jurassic Park (1993)',
 'Twelve Monkeys (1995)',
 'Men in Black (1997)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Hunt for Red October, The (1990)',
 'Speed (1994)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Gattaca (1997)']

In [30]:
def get_similar_movie(movie_name: str):
    title = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(title)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("please")

please


In [31]:
get_similar_movie('Star Wars: Episode IV - A New Hope (1977)')

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Alien (1979)',
 'Terminator, The (1984)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Back to the Future (1985)',
 'Matrix, The (1999)',
 'Indiana Jones and the Last Crusade (1989)']

In [34]:
get_similar_movie('My Fair Lady (1964)')

['My Fair Lady (1964)',
 'West Side Story (1961)',
 'King and I, The (1956)',
 "Singin' in the Rain (1952)",
 'Sound of Music, The (1965)',
 'Oliver! (1968)',
 'Romeo and Juliet (1968)',
 'American in Paris, An (1951)',
 'Gigi (1958)',
 'South Pacific (1958)']

In [35]:
# 영화 추천하기

user = user_to_idx['kris']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(107, 0.7077596),
 (62, 0.6415631),
 (14, 0.4172082),
 (44, 0.41572383),
 (64, 0.39925277),
 (141, 0.38189775),
 (117, 0.36881295),
 (375, 0.35645604),
 (145, 0.35295844),
 (193, 0.34288883),
 (354, 0.33665812),
 (651, 0.33255872),
 (5, 0.3168584),
 (685, 0.29762685),
 (194, 0.29016456),
 (663, 0.28087682),
 (882, 0.2751035),
 (236, 0.26864627),
 (680, 0.26201832),
 (87, 0.2613421)]

In [38]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Jurassic Park (1993)',
 'Total Recall (1990)',
 'Sound of Music, The (1965)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Fugitive, The (1993)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Face/Off (1997)',
 'Fifth Element, The (1997)',
 'Alien (1979)',
 'West Side Story (1961)',
 'Aliens (1986)',
 'Princess Bride, The (1987)',
 'Planet of the Apes (1968)',
 'Die Hard (1988)',
 "Singin' in the Rain (1952)",
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Speed (1994)',
 'Blade Runner (1982)',
 'Braveheart (1995)']

In [41]:
# explain 매서드를 이용한 기여도 확인

Total_Recall = movie_to_idx['Total Recall (1990)']
explain = als_model.explain(user, csr_data, itemid=Total_Recall)

In [42]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Terminator 2: Judgment Day (1991)', 0.19678885746187996),
 ('Men in Black (1997)', 0.17347186117403587),
 ('Matrix, The (1999)', 0.1476075198223057),
 ('Terminator, The (1984)', 0.11309621228673741),
 ('My Fair Lady (1964)', -0.0024930469373746406)]

## 회고

  - 첫번째로 여태까지 NLP 관련 노드들은 어렵기만하고 재미가 없었는데 이번 추천하는 모델을 만드는 노드는 생각보다 재미있었다.
  - 그럴만한 이유가 이번 데이터셋은 내가 따로 전처리를 많이 할 필요 없이 깔끔하게 가공된 데이터를 제공해주었기 때문인 것 같다.
  - 시작할때 연습 노드의 파일은 이미 데이터 프레임이 하나로 되어 있었지만 우리가 진행하려는 데이터셋은 프레임이 나뉘어져있어서 그것을 합치는데 이것저것 찾아보느라고 시간이 꽤 걸렸다.
  - 최근 fundamental node에서도 추천 관련 내용들이 나오고 그것들을 한번 본 이후에 이번 노드를 진행해서 주제가 크게 어렵게 느껴지지는 않았던 것 같다.
  - 하지만 아직까지도 데이터 전처리부분은 너무 어려운 부분으로 다가온다.
  - 어떤 부분들을 처리해줘야 하는지, 실제로 모델에 넣어서 사용하기 위해서는 어떤 칼럼들이 중요하고 어떤 칼럼들은 필요가 없는지 등등의 전처리 과정들이 아직도 생각보다 어렵다.
  - 그리고 사용하는 모델이 어떻게 구현되고 어떤 방식을 통해서 작업이 이루어지는지 자세한 모델 구현 과정까지는 이해하지 못했다.
  - 이번 노드에서는 이런 영화 추천 및 여러가지 추천에 있어서 이런 모델을 사용하면 되겠다라는 정도의 지식을 얻은 것 같다.

  - 만든 모델에 있어서도 모델이 생각보다 좋은 성능을 보인 것 같다.
  - 성능이 좋지 않으면 하이퍼 파라미터(factors, iterations)에 변화를 줘서 추가 훈련을 진행하고 결과를 받아보려 했지만 처음부터 괜찮은 성능이 나와준것 같다.
  - 정확한 수치적인 통계는 잘 모르겠는데 처음 입력했던 선호 영화들과 비교했을때 비교적 비슷한 느낌의 영화들이 추천된 것을 확인할 수 있었다.

  - 추후에 복습을 진행할 때 영화 데이터셋 말고도 구할 수 있는 다른 데이터셋을 가져다가 전처리 및 모델 구성을 추가적으로 해봐야 할 것 같다.